# **Семинар 4 — Контейнеризация ML-сервиса с помощью Docker**

## **Цель занятия**

На этом семинаре мы освоим принципы упаковки ML-сервиса в Docker-контейнер:  
создание `Dockerfile`, работа с зависимостями и переменными окружения,  
проверка запуска контейнера, базовые практики оптимизации и подготовка к развёртыванию в облаке или Kubernetes.

---

## **План занятия**

1. Повторение структуры ML-сервиса (REST/gRPC)  
2. Создание `Dockerfile`  
3. Работа с зависимостями и кешированием слоёв  
4. Добавление `ENTRYPOINT`, переменных окружения, `health-check`  
5. Сборка и запуск контейнера  
6. Проверка эндпоинтов `/health` и `/predict`  
7. Мини-оптимизация образа (размер, безопасность)  
8. Обсуждение типичных ошибок и CI/CD-интеграции


In [ ]:
# Проверим наличие Docker
!docker --version || echo "❌ Docker не установлен (в Colab можно показать только пример)"

## **1. Подготовка проекта**

Убедимся, что структура ML-сервиса готова к упаковке.

Сервис должен корректно запускаться локально. Сервис собирали в семинаре 3

## **2. Создание Dockerfile**

Минимальный пример для Python-сервиса на базе `python:3.11-slim`.

> **Основные принципы:**
> - Используем официальный минимальный базовый образ.  
> - Указываем переменные окружения (`ENV`).  
> - Устанавливаем зависимости с опцией `--no-cache-dir`.  
> - Копируем проект и задаём `CMD` для запуска сервера.


In [ ]:
dockerfile_content = """
# Базовый образ
FROM python:3.11-slim

# Переменные окружения
ENV PORT=50051 \\
    MODEL_PATH=/app/models/model.pkl \\
    MODEL_VERSION=v1.0.0

# Рабочая директория
WORKDIR /app

# Копирование зависимостей и установка
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Копируем весь проект
COPY . .

# Открываем порт
EXPOSE 50051

# Точка входа
CMD ["python", "-m", "server.server"]
"""

with open("ml_grpc_service/Dockerfile", "w") as f:
    f.write(dockerfile_content.strip())

print("✅ Dockerfile создан:")
!cat ml_grpc_service/Dockerfile


## **3. Сборка и запуск контейнера**

Собираем образ и запускаем контейнер:

docker build -t ml-grpc-service .

docker run -p 50051:50051 ml-grpc-service

Проверяем клиентом из прошлой лабораторной:

python -m client.client



## **4. Добавление `.dockerignore`**

In [ ]:
dockerignore = """
__pycache__/
*.pyc
.git
.venv
tests/
"""

with open("ml_grpc_service/.dockerignore", "w") as f:
    f.write(dockerignore.strip())

print("✅ .dockerignore создан:")
!cat ml_grpc_service/.dockerignore


## **5. Проверка состояния контейнера**

После запуска можно использовать команды диагностики:



## **7. Использование docker-compose (опционально)**

Для более сложных систем можно собрать `docker-compose.yml`:

version: "3.9"

services:

grpc_service:

build: .

ports:

- "50051:50051"

environment:

- MODEL_PATH=/app/models/model.pkl

- MODEL_VERSION=v1.0.0

restart: always

## **8. Проверка воспроизводимости (reproducibility)**

Пересоберите образ на другой машине или в облаке:  
результаты `/health` и `/predict` должны быть идентичны.

Это подтверждает, что окружение полностью воспроизводимо —  
ключевой принцип MLOps.


## **9. Типичные ошибки**

| Ошибка | Причина | Решение |
|--------|----------|---------|
| `Module not found` | Неправильный `WORKDIR` или структура проекта | Убедитесь, что корень совпадает с `/app` |
| `Permission denied` | Запуск от непривилегированного пользователя без прав | Настройте `USER` и права доступа |
| Порт не слушается | Несоответствие `EXPOSE` и `docker run -p` | Проверьте порты |
| Модель не найдена | Неправильный `MODEL_PATH` | Проверьте путь и COPY models/ |
| Слишком тяжёлый образ | Кеши и временные файлы в слоях | Используйте `--no-cache-dir`, `.dockerignore`, slim-образы |


## **10. Мини-задание по семинару**

1. Добавьте в `Dockerfile` переменную `MODEL_VERSION` и выведите её в ответе `/health`.  
2. Реализуйте multi-stage-сборку, уменьшив размер образа минимум на 40 %.  
3. Проверьте, что после перезапуска контейнера модель не теряется.  
4. Измерьте время старта контейнера и латентность `/predict` (через `time` или `grpcurl`).


## **11. Итоги**

- Контейнеризация обеспечивает переносимость и воспроизводимость ML-сервисов.  
- `Dockerfile` описывает все шаги создания среды и запуска.  
- Лучшие практики: минимальные образы, переменные окружения, `health-check`.  
- Полученный контейнер готов к CI/CD и развёртыванию в Kubernetes.


## **12. Вопросы для самопроверки**

1. Что делает команда `docker build -t ...` и как связаны слои образа?  
2. Зачем нужен `.dockerignore` и как он влияет на кеширование?  
3. Почему важно задавать переменные окружения через `ENV`, а не жёстко в коде?  
4. Как работает multi-stage-сборка и зачем она нужна?  
5. Как проверить, что контейнер действительно слушает нужный порт?
